In [12]:
import numpy as np
import time
# 对比运行时间
def list_and_numpy():
    t0 = time.time()
    # python list
    l = list(range(100))
    for _ in range(10000):
        for i in range(len(l)):
            l[i] += 1
    t1 = time.time()
    # numpy array
    a = np.array(l)
    for _ in range(10000):
        a += 1
    print("Python list spend {:.3f}s".format(t1 - t0))
    print("Numpy array spend {:.3f}s".format(time.time() - t1))
list_and_numpy()

Python list spend 0.029s
Numpy array spend 0.005s


In [13]:
import numpy as np
import time
n = 10000000
# 场景1：预分配NumPy数组
start = time.time()
state_array = np.zeros((n, 3))  # 假设每个状态是3维向量
for i in range(n):
    state_array[i] = np.array([i*0.1, i*0.2, i*0.3])  # 模拟状态赋值
array_time = time.time() - start

# 场景3：动态扩展Python列表
start = time.time()
state_list = []
for i in range(n):
    state_list.append([i*0.1, i*0.2, i*0.3])  # 模拟状态赋值
list1_time = time.time() - start

# 场景3：动态扩展初始化的Python列表
start = time.time()
state_list = [None] * n # 预分配列表空间
for i in range(n):
    state_list[i] = [i*0.1, i*0.2, i*0.3]  # 模拟状态赋值
list2_time = time.time() - start

print(f"预分配NumPy数组耗时：{array_time:.6f}秒")
print(f"动态扩展列表耗时：{list1_time:.6f}秒")
print(f"动态扩展初始化的列表耗时：{list2_time:.6f}秒")
print(f"NumPy比动态扩展列表快 {list1_time/array_time:.2f} 倍")
print(f"NumPy比动态扩展初始化的列表快 {list2_time/array_time:.2f} 倍")

预分配NumPy数组耗时：4.983861秒
动态扩展列表耗时：4.257731秒
动态扩展初始化的列表耗时：3.786361秒
NumPy比动态扩展列表快 0.85 倍
NumPy比动态扩展初始化的列表快 0.76 倍
